In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers
from keras import models
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
from tensorflow import keras
import os
from scipy import ndimage

#### Data Augumentation

In [3]:
def data_augmentation(images, labels):
    aug_images = []
    aug_labels = []    
    
    for x, y in zip(images, labels):        
        aug_images.append(x)
        aug_labels.append(y)
        
        bg_value = np.median(x)
        
        for _ in range(6):
            angle = np.random.randint(-15, 15, 1)            
            rot_img = ndimage.rotate(x, angle[0], reshape=False, cval=bg_value)
            
            shift = np.random.randint(-2, 2, 2)
            shift_img = ndimage.shift(rot_img, shift, cval=bg_value)            
            
            aug_images.append(shift_img)
            aug_labels.append(y)
            
    aug_images = np.array(aug_images)
    aug_labels = np.array(aug_labels)
    
    return aug_images, aug_labels

In [4]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'minst_cnn_best'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

#### Data PipeLine -- 1

In [5]:
# TRAIN
train = pd.read_csv('./data/train.csv')
train_digit = train['digit'].values
train_letter = train['letter'].values
train_img = train.iloc[:,3:].values.reshape(-1,28,28)



# TEST
test = pd.read_csv('./data/test.csv')
test_img = test.iloc[:,2:].values.reshape(-1, 28, 28, 1).astype(np.int)
X_test_letter = test['letter'].values

answer = pd.read_csv('./data/team_answer.csv')

y = train['digit']


test_img = test.drop(['id', 'letter'], axis=1).values
test_img = test_img.reshape(-1, 28, 28, 1)
test_img = test_img/255

output = pd.read_csv('./data/submission.csv')

#### 파라미터

In [13]:
learning_rate = 0.001
training_epochs = 6
batch_size = 100

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(train_img, y, test_size = 0.2)
X_train, Y_train = data_augmentation(X_train, Y_train)

X_train = X_train.astype(np.float32) / 255.
X_test = X_test.astype(np.float32) / 255.
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)


Y_train = to_categorical(Y_train, 10)
Y_test = to_categorical(Y_test, 10) 

train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(
                buffer_size = 10000).batch(batch_size)
test_data = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(batch_size)

In [8]:
class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME'):
        super(ConvBNRelu, self).__init__()
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, 
                                        padding=padding, kernel_initializer='he_normal')
        self.batchnorm = tf.keras.layers.BatchNormalization()
    def call(self, inputs, training=False):
        layer = self.conv(inputs)
        layer = self.batchnorm(layer)
        layer = tf.nn.relu(layer)
        return layer

In [9]:
class DenseBNRelu(tf.keras.Model):
    def __init__(self, units):
        super(DenseBNRelu, self).__init__()
        self.dense = keras.layers.Dense(units=units, kernel_initializer='he_normal')
        self.batchnorm = tf.keras.layers.BatchNormalization()
    def call(self, inputs, training=False):
        layer = self.dense(inputs)
        layer = self.batchnorm(layer)
        layer = tf.nn.relu(layer)
        return layer

In [14]:
class Create_Model(tf.keras.Model):
    def __init__(self):
        super(Create_Model, self).__init__()
        self.conv1 = ConvBNRelu(filters=32, kernel_size=[3, 3], padding='SAME')        
        self.pool1 = keras.layers.MaxPool2D(padding='SAME')
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], padding='SAME')
        self.pool2 = keras.layers.MaxPool2D(padding='SAME')
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3], padding='SAME')
        self.pool3 = keras.layers.MaxPool2D(padding='SAME')
        self.pool3_flat = keras.layers.Flatten()
        self.dense4 = DenseBNRelu(units=256)
        self.drop4 = keras.layers.Dropout(rate=0.4)
        self.dense5 = keras.layers.Dense(units=10, kernel_initializer='glorot_normal')
    def call(self, inputs, training=False):
        net = self.conv1(inputs)        
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.pool3(net)
        net = self.pool3_flat(net)
        net = self.dense4(net)
        net = self.drop4(net)
        net = self.dense5(net)
        return net

In [15]:
models = []
num_models = 5
for m in range(num_models):
    models.append(Create_Model())

In [16]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(
        y_pred=logits, y_true=labels, from_logits=True))    
    return loss

In [17]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

In [18]:
def evaluate(models, images, labels):
    predictions = np.zeros_like(labels)
    for model in models:
        logits = model(images, training=False)
        predictions += logits
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

#### LearningRate Decay

In [24]:
lr_decay = tf.keras.optimizers.schedules.ExponentialDecay(learning_rate,
                                                          X_train.shape[0]/batch_size*num_models*5,
                                                          0.5, staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_decay)

In [25]:
checkpoints = []
for m in range(num_models):
    checkpoints.append(tf.train.Checkpoint(cnn=models[m]))

In [26]:
class Create_Model2(tf.keras.Model):
    def __init__(self):
        super(Create_Model2, self).__init__()
        self.conv0 = ConvBNRelu(filters=32, kernel_size=[3, 3], padding='SAME')        
        self.conv1 = ConvBNRelu(filters=32, kernel_size=[3, 3], padding='SAME')        
        self.pool1 = keras.layers.MaxPool2D(padding='SAME')
        
        self.drop1 = keras.layers.Dropout(rate=0.3)
        
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], padding='SAME')        
        self.conv3 = ConvBNRelu(filters=64, kernel_size=[3, 3], padding='SAME')        
        self.conv4 = ConvBNRelu(filters=64, kernel_size=[3, 3], padding='SAME')
        self.pool4 = keras.layers.MaxPool2D(padding='SAME')
        
        self.drop4 = keras.layers.Dropout(rate=0.3)
        
        self.conv5 = ConvBNRelu(filters=128, kernel_size=[3, 3], padding='SAME')
        self.conv6 = ConvBNRelu(filters=128, kernel_size=[3, 3], padding='SAME')
        self.conv7 = ConvBNRelu(filters=128, kernel_size=[3, 3], padding='SAME')
        self.pool7 = keras.layers.MaxPool2D(padding='SAME')
        
        self.drop7 = keras.layers.Dropout(rate=0.3)
        
        self.conv8 = ConvBNRelu(filters=256, kernel_size=[3, 3], padding='SAME')
        self.conv9 = ConvBNRelu(filters=256, kernel_size=[3, 3], padding='SAME')
        self.conv10 = ConvBNRelu(filters=256, kernel_size=[3, 3], padding='SAME')
        self.pool10 = keras.layers.MaxPool2D(padding='SAME')   
        
        self.pool10_flat = keras.layers.Flatten()
        self.dense11 = DenseBNRelu(units=256)
        
        self.drop12 = keras.layers.Dropout(rate=0.5)
        
        self.dense13 = keras.layers.Dense(units=10, kernel_initializer='glorot_normal')
    def call(self, inputs, training=False):
        net = self.conv0(inputs)
        net = self.conv1(net)
        net = self.pool1(net)
        net = self.drop1(net)
                         
        net = self.conv2(net)
        net = self.conv3(net)
        net = self.conv4(net)
        net = self.pool4(net)
        net = self.drop4(net)
                         
        net = self.conv5(net)
        net = self.conv6(net)
        net = self.conv7(net)
        net = self.pool7(net)
        net = self.drop7(net)
                         
        net = self.conv8(net)
        net = self.conv9(net)
        net = self.conv10(net)
        net = self.pool10(net)

        net = self.pool10_flat(net)
        net = self.dense11(net)
        net = self.drop12(net)
        net = self.dense13(net)
        return net

In [27]:
models = []
num_models = 5
for m in range(num_models):
    models.append(Create_Model2())

In [28]:
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0        
    for images, labels in train_data:
        for model in models:
            grads = grad(model, images, labels)                
            optimizer.apply_gradients(zip(grads, model.trainable_variables))            
            loss = loss_fn(model, images, labels)
            avg_loss += loss / num_models
        acc = evaluate(models, images, labels)
        avg_train_acc += acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_data:        
        acc = evaluate(models, images, labels)        
        avg_test_acc += acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
    
    
    for idx, checkpoint in enumerate(checkpoints):
        checkpoint.save(file_prefix=checkpoint_prefix+'-{}'.format(idx))

print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 1 loss = 1.96192145 train accuracy =  0.1300 test accuracy =  0.1020
Epoch: 2 loss = 0.99880451 train accuracy =  0.3064 test accuracy =  0.5980
Epoch: 3 loss = 0.63695079 train accuracy =  0.7735 test accuracy =  0.8880
Epoch: 4 loss = 0.46527433 train accuracy =  0.8828 test accuracy =  0.8880
Epoch: 5 loss = 0.36735347 train accuracy =  0.9288 test accuracy =  0.9020
Epoch: 6 loss = 0.26305613 train accuracy =  0.9621 test accuracy =  0.9140
Learning Finished!


### Prediction

In [190]:
test_size = len(test_img)
predictions = np.zeros(test_size * 10).reshape(test_size,10)

for m_idx, m in enumerate(models):
    p = m.predict(test_img)
    predictions += p
    
pred = np.zeros(10)

pred = np.argmax(predictions, axis = 1)

output['digit'] = pred

output.to_csv('Output.csv', index = False)